In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil
from io import BytesIO

#import cv2
from PIL import Image
import time
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [2]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80","%EA%B3%B5%EC%A0%95%EC%9C%84","%EA%B3%B5%EC%A0%95%EB%AC%B8%EA%B5%AC"
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8","쉬즈블로그","%EC%89%AC%EC%A6%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "http://echoblog.net/images/sponsor-banner.png","%EB%AC%B4%EC%83%81%EC%A0%9C%EA%B3%B5",
                      "sponsor","banner","echoblog","reviewplace","www.storyn.kr","pick-me.kr"
                      , "%EC%A0%9C%ED%92%88%EC%A0%9C%EA%B3%B5"
                      , "reviewplace"
                      ,"site-tracker.08liter.com"
                      ,"08liter"]
# %EC%95%94%ED%96%89%EC%96%B4%EC%82%AC -> 암행어사...는 좀 일반명사 같아서 고민중
# %EB%B0%B0%EB%84%88 -> 배너 제외..

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 4
epochs = 10
IMG_HEIGHT = 120
IMG_WIDTH = 240


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [5]:
#load model
checkpoint_path = "model_0728_v3(120_240)/cp-0010.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)

model_0728_v3(240_480)/cp-0020.ckpt


In [6]:
def convertToJpeg(im):
    with BytesIO() as f:
        new_img = im.convert("RGB")
        new_img.save(f, format='JPEG')
        return f.getvalue()


In [7]:
def PredictionWithoutDownload(uri):

    start_time = time.time();
    filename = uri.split('/')[-1].split('?')[0];
    img_url2 = urllib.parse.quote_plus(str(filename))
    img_url_final = uri.replace(uri.split('/')[-1].split('?')[0],img_url2)
    
#     response = requests.get(uri)
    response = urllib.request.urlopen(img_url_final).read()
#     print("---[1] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
#     img = Image.open(BytesIO(response.content))
    img = Image.open(BytesIO(response))
    
    if(img.format == "PNG") or (img.format == "GIF"):
        print("this image is",img.format,"type. could be change to jpg")
        png_to_jpg = convertToJpeg(img)
        img = Image.open(BytesIO(png_to_jpg))
#         img.show()
#     print("---[2] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #width,height
    resize_img = img.resize((IMG_WIDTH, IMG_HEIGHT))
#     print("---[3] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();

    input_arr = tf.keras.preprocessing.image.img_to_array(resize_img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
#     print("---[4] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    
    predictions_class = model.predict_classes(input_arr)
#     print("---[5] {}s seconds---".format((time.time()-start_time),".2f"))

    print("========pridiction[Without Download]========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    if(predictions_class == [[0]]):
        predictions = model.predict(input_arr)
        print("predictions : ",predictions);

        print("this is fake blog")
        saveImageVol2(uri,predictions)
#     print("predictions : " , predictions)
    return predictions_class


In [8]:
# =====================================================================
#이미지 저장만을 위한 함수
def saveImageVol2(img_url,predict):
    list = os.listdir("ad_img")
    last_index = len(list)
#         print("last_index : ",last_index)
    dirPath = 'ad_img'
    imgPath = f'ad_img/{last_index}_{predict}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
    except Exception as ex:
        print("img save error : " , ex)
        return False

In [9]:
# =====================================================================
#이미지 분류 [my model]

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
#    print("input_arr type : " ,type(input_arr))
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

    print("========pridiction========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    print("predictions : " , predictions);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class,predictions;

# =====================================================================
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#         print("img_url_final : " ,img_url_final);
        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        my_prediction, score  = imageClassification(imgPath);
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            if(imgPath.find("fake_") > 0):  #모델, url 둘다 이미지로 판단
                os.replace(imgPath, f'ad_img/url_model_{last_index}_{score}.jpg')
            else: #모델만 이미지로 판단
                os.replace(imgPath, f'ad_img/model_{last_index}_{score}.jpg')
            with open("fake_img_url_list.txt", "a") as file:
                file.write(f'model_{last_index}_{score}.jpg\n')
                file.write(f'{img_url_final}\n')
                file.write("================================\n")
            return False
        else: # url만 이미지로 판단
            list = os.listdir("not_fake")
            last_index = len(list)
            if(imgPath.find("fake_") > 0):
                list = os.listdir("ad_img")
                last_index = len(list)
                os.replace(imgPath, f'not_fake/url_{last_index}_{score}.jpg')
            else: #모델, url 둘다 이미지로 판단
                if(score <= 1500):  #confidence가 1500이하일때만 저장 
                    os.replace(imgPath, f'not_fake/url_model_{last_index}_{score}.jpg')
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
    
# =====================================================================
# image gethering
def findBlogImgGethering(searchQuery, searchOption, page):
    try:
        start_time = time.time();

        url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
        html = requests.get(url)
        # 1차, blog URL 찾기
        soup = BeautifulSoup(html.text, 'html.parser')
        index = 0;

        url_list = [];
        for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
            child_item = li_item.find('a',{'class' : 'sh_blog_title'})
            title = child_item.attrs['title']
            href = child_item.attrs['href']

            #url list 저장
            url_list.append(href)

            print(f'title : {title} , href : {href}')
            print('----------------------------------')

        # url 파싱 및 예외처리
        for url_item in url_list:
            #blog.me, blog.naver 같은 형태일때만 digging!
            if(url_item.find("blog")<0):
                continue;
                
            nickName = "";
            postNumber = "";

            if(url_item.find("blog.me") > 0):
                parsing = url_item;
                parsing = parsing.replace("https://","")
                nickName = parsing.split('.')[0]
                postNumber = parsing.split('/')[1]
                blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

            else:
                blogUrl = url_item.replace("https://", "https://m.");
            try:
                nickName = blogUrl.split('/')[-1].split('?')[0]
                postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]
            except:
                print("url parsing error : " , blogUrl);
                continue
            
            if nickName is "":
                nickName = "unknown"
            if postNumber is "":
                postNumber = "1"

            print("------ digging more -------")
            print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
            print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
            print("blogUrl : " , blogUrl)        
            start_time = time.time();
            #전체 목록 순회
            blog_html = requests.get(blogUrl)
            blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
            blog_image_class = blog_soup.find_all('img',{'class' : 'se-image-resource'});

            for div_obj in blog_image_class:
#                 for idx, img_item in enumerate(div_obj.find_all('img')):
                img_url = div_obj.attrs['src'];

                if(img_url.find("w80_blur") > 0):
                    img_url = img_url.replace("w80_blur" , "w800")
                    #np array after save image(for crawling data)
                isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                #np arr from img_url for validation
#                     PredictionWithoutDownload(img_url)
                if(isSaveSuccess):  
                    index = index+1
    #         print("---{}s seconds---".format(time.time()-start_time),".2f")
    except Exception as ex:
        print("error catch : " , ex)
        print("url : " ,img_url)


In [10]:
searchQuery = "강남역 맛집"
searchOption = 'sim'  #sim or date  
page = 1
end_page = 10
for i in list(range(page,end_page)):
    if i>1:
        page = page + 10;
    findBlogImgGethering(searchQuery,searchOption, page)
# findBlogImgGethering(searchQuery,searchOption,page)

title : 독특한 강남역 맛집 , href : https://candypowder.blog.me/222039456440
----------------------------------
title : 색다르던 강남역 맛집 , href : https://blog.naver.com/celinicious?Redirect=Log&logNo=222040018001
----------------------------------
title : 의미있었던 강남역 맛집 , href : https://blog.naver.com/bonbon_mikan?Redirect=Log&logNo=222033279409
----------------------------------
title : 즐거웠던 강남역 맛집 , href : https://blog.naver.com/crom2481?Redirect=Log&logNo=222038783092
----------------------------------
title : 빼어났던 강남역 맛집 , href : https://blog.naver.com/woandos?Redirect=Log&logNo=221999144394
----------------------------------
title : 탑오브탑 강남역 맛집 , href : https://blog.naver.com/boru130?Redirect=Log&logNo=222040885567
----------------------------------
title : 인상깊었던 강남역 맛집 , href : https://estf2846.blog.me/222036174327
----------------------------------
title : 가성비 좋은 강남역  맛집 , href : https://blog.naver.com/hyeminkbs?Redirect=Log&logNo=222041364296
----------------------------------
title : 남달랐던 강남

========pridiction========
predictions class :  [[0]]
predictions :  [[-821.5655]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-774.79126]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2911.0522]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1484.16]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1519.412]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-3041.841]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1761.2169]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-945.20294]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-355.53998]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-743.25824]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2757.697]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-767.2216]]

========pridiction========
predictions class :  [[0]]
predictions :  [[-202.60768]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1186.2883]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1013.8434]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1686.2037]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-562.40955]]
------ digging more -------
nickName :  estf2846
postNumber :  222036174327
blogUrl :  https://m.blog.naver.com/estf2846?Redirect=Log&logNo=222036174327
========pridiction========
predictions class :  [[0]]
predictions :  [[-1845.3986]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1873.2837]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2845.4346]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1501.6218]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2611.7449]]
=======

========pridiction========
predictions class :  [[0]]
predictions :  [[-540.37036]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-4390.197]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-4107.881]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2148.0955]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1224.8055]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2614.9438]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1362.3915]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2561.456]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2989.6638]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-860.45386]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2146.031]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1251.141

========pridiction========
predictions class :  [[0]]
predictions :  [[-2627.6504]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1362.9054]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1618.303]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1649.3956]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1869.5399]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1959.4092]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1122.5706]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2021.6333]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1634.4226]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-892.1111]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-204.7621]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-750.739

...

========pridiction========
predictions class :  [[0]]
predictions :  [[-1443.5201]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1422.129]]
========pridiction========
predictions class :  [[1]]
predictions :  [[463.51782]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-850.6725]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2367.1274]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-614.5745]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-128.85196]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2427.1982]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1613.9559]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2174.6523]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2470.8987]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1334.904

========pridiction========
predictions class :  [[0]]
predictions :  [[-1596.1864]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2717.8123]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1509.479]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1160.6107]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1319.9735]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1646.8231]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2134.815]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1060.3612]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1094.614]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1269.0533]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1299.1971]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2180.08

========pridiction========
predictions class :  [[0]]
predictions :  [[-771.564]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1134.8185]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1028.3574]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1643.2278]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2064.6873]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2097.327]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1899.3142]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-630.70984]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1051.3787]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-633.1766]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-149.97517]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-406.5776

========pridiction========
predictions class :  [[0]]
predictions :  [[-3088.1775]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-3223.3328]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1921.973]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2370.576]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1272.9678]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2902.6287]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-902.7834]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2304.4487]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2702.658]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-2707.6404]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1485.5017]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-982.5498

KeyboardInterrupt: 

In [ ]:
# # blog gethering test
# def findBlogListTest(searchQuery, searchOption, page):
#     try:
#         print(f'\n###############{page}###############\n');
#         url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
#         html = requests.get(url)
#         # 1차, blog URL 찾기
#         soup = BeautifulSoup(html.text, 'html.parser')
#         index = 0;

#         url_list = [];
#         for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
#             child_item = li_item.find('a',{'class' : 'sh_blog_title'})
#             title = child_item.attrs['title']
#             href = child_item.attrs['href']

#             #url list 저장
#             url_list.append(href)

#             print(f'title : {title} , href : {href}')
#             print('----------------------------------')

#     except Exception as ex:
#         print("error catch : " , ex)
#         print("url : " ,img_url)

In [ ]:
# searchQuery = "서울대입구역 하남돼지집"
# searchOption = 'sim'  #sim or date  
# page = 1
# end_page = 11
# # 1->11->21->순으로
# for i in list(range(page,end_page)):
#     if i>1:
#         page = page + 10;
#     findBlogListTest(searchQuery,searchOption,page)

In [ ]:
# def search_function(tag):
#     return tag.attr('class') == 'se-imageStrip' or tag.attr('class') == 'se-image';

# #이미지 여러개 합쳐진 문제 
# url = 'https://m.blog.naver.com/danee_blog?Redirect=Log&logNo=221516896081'

# blog_html = requests.get(url)
# blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
# # blog_image_class = blog_soup.find_all('div',{'class' : 'se-image', 'class' : 'se-imageStrip'});
# blog_image_class = blog_soup.find_all('img',{'class' : 'se-image-resource'});

# for div_obj in blog_image_class:
#     img_url = div_obj.attrs['src'];
#     print(img_url)
# #     for idx, img_item in enumerate(div_obj.find_all('img')):
# #         img_url = img_item.attrs['src'];
# #         print(img_url)
# #         if(img_url.find("w80_blur") > 0):
# #             img_url = img_url.replace("w80_blur" , "w800")
#             #np array after save image(for crawling data)
# #         isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
#         #np arr from img_url for validation
#     #                     PredictionWithoutDownload(img_url)
# #         if(isSaveSuccess):  
# #             index = index+1
            
# # print("blog_image_class : " , blog_image_class.length)